In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pymongo import MongoClient
from bson.json_util import dumps
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "CmS1201"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # Creation of an anchor for the logo
    # Make image an href to the desired website, www.snhu.edu
    # Open link in a new tab by setting a blank target
    html.A([
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                            height = 250, width = 251))], href = 'https://www.snhu.edu', target = "_blank"),
    html.Center(html.B(html.H1('Corey Sampsons\' CS-340 Dashboard'))),
    html.Hr(),
    
    # Creation of radio buttons to act as a filter
    # Default set to "All"
    html.Div(
        dcc.RadioItems(
            id = 'filter-type',
            options = [
                {'label' : 'All', 'value' : 'All'},
                {'label' : 'Water Rescue', 'value' : 'Water'},
                {'label' : 'Mountain or Wilderness Rescue', 'value' : 'Mountain'},
                {'label' : 'Disaster Rescue or Individual Tracking', 'value' : 'Disaster'},
            ]
        )
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[
                             {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
                         ],
                         data=df.to_dict('records'),
                         editable=True,
                         row_selectable="single", # Allow for single selectable row
                         selected_rows=[],
                         filter_action="native", # Allow for filtering
                         sort_action="native", # Allow for sorting
                         page_action="native", # Enable pagination
                         page_current=0, # Set start page
                         page_size=10, # Set rows per page
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                 html.Div([
                     dcc.Dropdown(
                         id='pie_dropdown',
                         className='col s12 m6',
                         options=[
                             {'label': 'Breed', 'value': 'breed'},
                             {'label': 'Age', 'value': 'age_upon_outcome'},
                             {'label': 'Outcome', 'value': 'outcome_type'}
                         ],
                         value = 'breed',
                         multi=False,
                         clearable=False
                         )
                 ],
                style = {"width":"10%"}
                 ),
                 
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                     style={"width":"45%"}
                 ),
                 
                 html.Div(
                     id='map-id',
                 className='col s12 m6',
                 style={"Width":"45%"}
                 )
             ])
])
#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback([Output('datatable-id','data'),
              Output('datatable-id','columns'),
              Output('datatable-id','selected_rows')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
    if filter_type == 'All':
        df = pd.DataFrame.from_records(db.read({}))
    elif filter_type == 'Water':
        df = pd.DataFrame(list(db.read({
            "animal_type":"Dog",
            "breed":{"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks":{"$gte": 26},
            "age_upon_outcome_in_weeks":{"$lte": 156}
        })))
    elif filter_type == 'Mountain':
        df = pd.DataFrame(list(db.read({
            "animal_type":"Dog",
            "breed":{"$in":["Germn Shepherd", "Alaskan Malamute", "Old English Sheepdog", 
                            "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte": 26},
            "age_upon_outcome_in_weeks":{"$lte": 156}
        })))
    elif filter_type == 'Disaster':
        df = pd.DataFrame(list(db.read({
            "animal_type":"Dog",
            "breed":{"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", 
                           "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte": 20},
            "age_upon_outcome_in_weeks":{"$lte": 300}
        })))
    else:
        df = pd.DataFrame.from_records(db.read({}))
        
    data = df.to_dict('records')
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    selected_rows = [0]
    
    return (data, columns, selected_rows)


# Changes color of a selected cell
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selectedRows):
    return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selectedRows]

# Callback for pie chart
# Set to plot all of the data across all the pages instead of viewable data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('pie_dropdown', 'value')])
def update_graphs(viewData, dropdownValue):
    
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(
        figure = px.pie(
            data_frame = dff,
            names=dropdownValue)
        )
    ]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(virtualRows):
    
    if not virtualRows:
        Array = (30.75, -97.48)
        toolTip = "Austin Animal Center (AAC)"
        popUpHeading = "Austin Animal Shelter"
        popUpParagraph = "Shelter Home Location"
        
    else:
        dff = pd.DataFrame(df.iloc[virtualRows])
        coordLat = float(dff['location_lat'].to_string().split()[1])
        coordLong = float(dff['location_long'].to_string().split()[1])
        Array = (coordLat, coordLong)
        
        toolTip = dff['breed']
        popUpHeading = "Animal Name"
        popUpParagraph = dff['name']
        
    return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=Array,
                  zoom=10, children=[dl.TileLayer(id="base-layer-id"),
                                    dl.Marker(position=Array, children=[
                                        dl.Tooltip(toolTip),
                                        dl.Popup([
                                            html.H1(popUpHeading),
                                            html.P(popUpParagraph)
                                        ])
                                    ])
                                    ])
           ]


app.run_server()

Dash app running on http://127.0.0.1:11420/
